# FileBox

The `FileBox` provides a Box-like interface to file objects.

**NOTE**: The [upstream `ipywidgets.FileUpload`](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#File-Upload) should probably be used instead of this class, as it might have more integrations. 

In [ ]:
from wxyz.html import FileBox, File, JSONFile

In [ ]:
import pandas as pd, ipywidgets as W, IPython as I, IPython.display as D, traitlets as T
from wxyz.core import UnJSON, JSON
from wxyz.lab import DockBox, DockPop, Editor
from pathlib import Path

## It's a box

In [ ]:
box = W.VBox()

In [ ]:
if __name__ == "__main__":
    D.display(DockPop([box], mode="split-right"))

In [ ]:
files = FileBox()
box.children = [W.HTML("<h2>It uploads files</h2>"), files]

In [ ]:
if __name__ == "__main__":
    D.display(files)

# What we know about files

In [ ]:
info = W.Output(layout=dict(width="100%"))

In [ ]:
def show(*_):
    info.clear_output()
    with info:
        display(pd.DataFrame([
            dict(
                name=c.name, 
                size=c.size, 
                mime_type=c.mime_type, 
                last_modified=pd.to_datetime(c.last_modified, unit="ms")
            )
            for c in files.children
        ]).T)

files.observe(show, "children")
show()

## Build an app

In [ ]:
app = DockBox([info, files], layout=dict(height="60vh"))
files.description = "File Upload"
info.description = "File Stats"

In [ ]:
box.children = [app]

## File

Unlike the upstream `FileUpload`, `FileBox` treats each file as an individual widget. They can be created kernel-side.

In [ ]:
name = "Filing.ipynb" if __name__ == "__main__" else __file__
this_path = Path(name)
this_bytes = this_path.read_bytes()
this_stat = this_path.stat()
a_file = File(
    name=this_path.name, 
    value=this_bytes,
    size=len(this_bytes),
    mime_type="application/x-ipynb+json",
    last_modified=int(1e3 * this_stat.st_mtime)
)

In [ ]:
files.children = [a_file]

## Proxy
Proxy files allow for working with format-aware editing, such as the `JSONFile`.

In [ ]:
editor = Editor(layout=dict(height="100%"), 
    config=dict(
        mode="application/ld+json", 
        theme="zenburn", 
        foldGutter=True, 
        lineNumbers=True,
        lineWrapping=True,
        gutters=["CodeMirror-linenumbers", "CodeMirror-foldgutter"],
        readOnly=True,
    ))
to_json = JSON()
un_json = UnJSON(indent=2)

edit_links = {}

def link_editor(*_):
    a_file = files.children[0]
    if Path(a_file.name).suffix in [".json", ".ipynb", ".jsonld"]:
        a_json_proxy = a_file.proxy(JSONFile)
        for linker, links in edit_links.items():
            for link in links:
                link.unlink()
        [edit_links.pop(k) for k in list(edit_links)]
        edit_links[a_json_proxy] = [
            W.jsdlink((a_json_proxy, "json"), (un_json, "source")),
            W.jsdlink((un_json, "value"), (editor, "value")),
            W.jsdlink((a_json_proxy, "name"), (editor, "description")),
        ]
        a_file.observe(show, "size")
files.observe(link_editor, "children")
link_editor()
files.accept = ["*.json", "*.ipynb", "*.jsonld"]
files.description = "JSON (and Friends) Viewer"

In [ ]:
app.children = [info, files, editor]
app.dock_layout = {'type': 'split-area',
 'orientation': 'horizontal',
 'children': [{'type': 'split-area',
   'orientation': 'vertical',
   'children': [{'type': 'tab-area', 'widgets': [1], 'currentIndex': 0},
    {'type': 'tab-area', 'widgets': [0], 'currentIndex': 0}],
   'sizes': [0.5, 0.5]},
  {'type': 'tab-area', 'widgets': [2], 'currentIndex': 0}],
 'sizes': [0.4409, 0.559]}